In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy

In [25]:
## load data
train = pd.read_csv('cleaned_data/TestSet_withpast3.csv')
test = pd.read_csv('sampleSubmission.csv')

In [26]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
#train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
#train['PinCode'].fillna(-1, inplace=True)
#train['DOB'].fillna("1", inplace=True)


In [7]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

KeyError: 'product_code'

In [ ]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

train.replace({"State": state_dict}, inplace=True)

In [ ]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(20)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

In [ ]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [ ]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [ ]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

In [27]:
train.dropna(inplace=True)

In [28]:
prod_dict = {'_'}

#train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_5','')
train.columns=train.columns.str.replace('_4','')
train.columns=train.columns.str.replace('_3','')
train.columns=train.columns.str.replace('_2','')
train.columns=train.columns.str.replace('_1','')
train.reset_index(drop=True).head()

,customerID,Gender,State,100105505.0,108000537.0,108000568.0,108000707.0,108001125.0,108001127.0,108003448.0,...,300808923.0,300840018.0,300942704.0,300974316.0,300974360.0,500096181.0,1000010411.0,1000058092.0,1000336252.0,1000609658.0
0,BBID_204975,male,MADHYA PRADESH,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BBID_204975,male,MADHYA PRADESH,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BBID_204975,male,MADHYA PRADESH,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BBID_204975,male,MADHYA PRADESH,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BBID_204975,male,MADHYA PRADESH,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
prod_dict = {'_'}
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

In [29]:
test = test[['customerID']]

In [30]:
true = deepcopy(train)

In [31]:
users = true['customerID'].tolist()
true.drop('customerID', axis=1, inplace=True)
true.drop('Gender', axis=1, inplace=True)
true.drop('State', axis=1, inplace=True)

items = true.columns.tolist()
u = {}
for i in range(len(users)):
    u[users[i]] = i

trueMat = np.array(true)
print("users dict formed")

users dict formed


In [32]:
def reorder(train):
    train.index = train['customerID'].tolist()
    train.drop('customerID', axis=1, inplace=True)
    train.drop('Gender', axis=1, inplace=True)
    train.drop('State', axis=1, inplace=True)
    train = train.reindex(users)
    return train

In [33]:
train = reorder(train)

In [34]:
def svd(train, trueMat, k):
    utilMat = np.array(train)

    mask = np.isnan(utilMat)
    masked_arr=np.ma.masked_array(utilMat, mask)
    item_means=np.mean(masked_arr, axis=0)
    utilMat = masked_arr.filled(item_means)

    x = np.tile(item_means, (utilMat.shape[0],1))

    utilMat = utilMat - x

    print(utilMat)

    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]

    s_root=sqrtm(s)

    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)

    UsV = UsV + x

    UsV = np.ma.masked_where(trueMat==1,UsV).filled(fill_value=-999)

    #print(UsV)
    print("svd done")
    return UsV

In [35]:
def max_items(UsV,x,j):
    out = []

    for xx in x:
        if UsV[j,xx]>0.001: # setting a threshold
            out.append(items[xx])
        
    if len(out) < 20:
        for j in range(20 - len(out)):
            out.append("None")
        

    return out




In [36]:
def recommend(test):

    UsV = (5*svd(train=train,trueMat=trueMat,k=13) + 5*svd(train=train, trueMat=trueMat, k=13))/10

    pred = []
    testusers = test['customerID'].tolist()

    n = 20
    for user in testusers[:]:
        if user not in u:
            pred.append("300776411,300776410,108037568,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None")
            test['products'] = pd.Series(pred)
        else:
            j = u[user]
            p = max_items(UsV, UsV[j,:].argsort()[-n:][::-1], j)
            pred.append(",".join(p))
            print(len(p))
            test['products'] = pd.Series(pred)
    #test['products'] = pred
    test.to_csv('sub_new_k13test_20prods_default3_to_max.csv', index=False)


In [37]:
recommend(test)

[[-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 ..., 
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]]
svd done
[[-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 ..., 
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]
 [-0.10063997281531405 0.0 0.0 ..., 0.0 0.0 0.0]]
svd done
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
